## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.88,147.33,55.99,62,75,10.29,broken clouds
1,1,Savantvadi,IN,15.90,73.82,75.20,100,90,6.93,mist
2,2,Atuona,PF,-9.80,-139.03,78.48,71,3,19.64,clear sky
3,3,Port Elizabeth,ZA,-33.92,25.57,57.20,71,6,9.17,clear sky
4,4,Vryburg,ZA,-26.96,24.73,50.65,43,0,6.80,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Savantvadi,IN,15.90,73.82,75.20,100,90,6.93,mist
2,2,Atuona,PF,-9.80,-139.03,78.48,71,3,19.64,clear sky
6,6,Ballina,AU,-28.87,153.57,75.99,77,6,3.36,clear sky
7,7,Lorengau,PG,-2.02,147.27,84.13,75,100,9.48,overcast clouds
8,8,Avarua,CK,-21.21,-159.78,77.00,78,25,3.36,light rain
11,11,Butaritari,KI,3.07,172.79,80.89,74,23,13.38,few clouds
13,13,Georgetown,MY,5.41,100.34,78.80,94,20,3.36,few clouds
18,18,Kapaa,US,22.08,-159.32,86.00,66,40,12.75,scattered clouds
25,25,Alofi,NU,-19.06,-169.92,78.80,78,75,24.16,broken clouds
29,29,Otatitlan,MX,18.20,-96.03,86.16,78,100,3.22,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                241
City                   241
Country                241
Lat                    241
Lng                    241
Max Temp               241
Humidity               241
Cloudiness             241
Wind Speed             241
Current Description    241
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Savantvadi,IN,15.90,73.82,75.20,100,90,6.93,mist
2,2,Atuona,PF,-9.80,-139.03,78.48,71,3,19.64,clear sky
6,6,Ballina,AU,-28.87,153.57,75.99,77,6,3.36,clear sky
7,7,Lorengau,PG,-2.02,147.27,84.13,75,100,9.48,overcast clouds
8,8,Avarua,CK,-21.21,-159.78,77.00,78,25,3.36,light rain
...,...,...,...,...,...,...,...,...,...,...
697,697,Coatzintla,MX,20.48,-97.45,86.00,66,5,11.41,clear sky
698,698,Vega De Alatorre,MX,20.03,-96.63,82.26,79,100,7.94,light rain
708,708,Mahajanga,MG,-15.72,46.32,77.00,83,40,4.70,scattered clouds
710,710,Port Keats,AU,-14.25,129.55,85.32,67,23,1.66,few clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Savantvadi,IN,75.20,mist,15.90,73.82,
2,Atuona,PF,78.48,clear sky,-9.80,-139.03,
6,Ballina,AU,75.99,clear sky,-28.87,153.57,
7,Lorengau,PG,84.13,overcast clouds,-2.02,147.27,
8,Avarua,CK,77.00,light rain,-21.21,-159.78,
11,Butaritari,KI,80.89,few clouds,3.07,172.79,
13,Georgetown,MY,78.80,few clouds,5.41,100.34,
18,Kapaa,US,86.00,scattered clouds,22.08,-159.32,
25,Alofi,NU,78.80,broken clouds,-19.06,-169.92,
29,Otatitlan,MX,86.16,light rain,18.20,-96.03,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
    # 6c. Get latitude and longitude from DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Savantvadi,IN,75.20,mist,15.90,73.82,Bungalow At Sawantwadi
2,Atuona,PF,78.48,clear sky,-9.80,-139.03,Villa Enata
6,Ballina,AU,75.99,clear sky,-28.87,153.57,Ramada Hotel & Suites by Wyndham Ballina Byron
7,Lorengau,PG,84.13,overcast clouds,-2.02,147.27,Lorengau Harbourside Hotel
8,Avarua,CK,77.00,light rain,-21.21,-159.78,Paradise Inn
11,Butaritari,KI,80.89,few clouds,3.07,172.79,Isles Sunset Lodge
13,Georgetown,MY,78.80,few clouds,5.41,100.34,Cititel Penang
18,Kapaa,US,86.00,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
25,Alofi,NU,78.80,broken clouds,-19.06,-169.92,Taloa Heights
29,Otatitlan,MX,86.16,light rain,18.20,-96.03,EL RANCHITO OTATITLAN HOTEL


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))